# Data Privacy Final Project
### Bradon Soucy and Sam Will

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
bioQuery = pd.read_csv("BioTIMEQuery_24_06_2021.csv", delimiter=',', encoding='ISO-8859-2', dtype={'PLOT': str}, usecols=[i for i in range(1,15)])
pd.set_option('display.max_columns', None)
bioQuery.head()

,STUDY_ID,DAY,MONTH,YEAR,SAMPLE_DESC,PLOT,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS,GENUS,SPECIES,GENUS_SPECIES
0,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,22,47.4,-95.12,1.0,0.0,Acer,rubrum,Acer rubrum
1,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,23,47.4,-95.12,3.0,0.0,Acer,saccharum,Acer saccharum
2,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,24,47.4,-95.12,1.0,0.0,Acer,spicatum,Acer spicatum
3,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,607,47.4,-95.12,12.0,0.0,Corylus,cornuta,Corylus cornuta
4,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Small,12,1911,47.4,-95.12,1.0,0.0,Populus,pinnata,Populus pinnata


In [18]:
# counts = bioQuery['YEAR'].value_counts()
tenBioQ = bioQuery.iloc[::10].copy()
cleanC = tenBioQ['YEAR'].value_counts()

smallC = cleanC[cleanC.values >= 1000]
# cRows = sum(smallC.values)
# smallCC = counts[counts.values <= 10]
# ccRows = sum(smallCC.values)

# # total 
# print(len(smallC))
# print("Rows included: " + str(cRows))
# print(len(smallCC))
# print("Rows included: " + str(ccRows))

# print(counts)
# print(cleanC)
# print(smallC)

In [19]:
clean = 0
count = 0
firstL = True
wfile = open("clean.csv", 'w', encoding='ISO-8859-2')
with open("BioTIMEQuery_24_06_2021.csv", encoding='ISO-8859-2') as file:
    for line in file:
        if not firstL:
            gen_spec = line.replace("\n","").replace('"','').split(',')[14]

            if count % 10 == 0 and gen_spec in smallC:
                clean +=1
                wfile.write(line)
        else:
            firstL = False
            wfile.write(line)
        count+=1
wfile.close()
print(clean)

KeyboardInterrupt: 

In [ ]:
cBioQ = pd.read_csv("clean.csv", delimiter=',', encoding='ISO-8859-2', dtype={'PLOT': str}, usecols=[i for i in range(1,15)])
print(len(cBioQ))
cBioQ.head()

In [ ]:
sortBioQ = cBioQ.sort_values(by='YEAR', ascending=False)

In [ ]:
plt.plot(sortBioQ['YEAR'], sortBioQ['sum.allrawdata.BIOMASS'])